In [55]:
import requests

from random import random
from time import sleep, time, ctime

All data comes from https://cryptorank.io/. It has API but there is no endpoint for me to target. So the data has to be parsed from the page.

In [6]:
headers = {
     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7,json", 
}


In [20]:
# Use generator to save time and memory
def my_generator(my_list):
    for item in my_list:
        yield item

In [ ]:
page_number = 0
funds_info = list()
while True:
    url = f"https://api.cryptorank.io/v0/funds?limit=100&offset={page_number}"
    response = requests.get(url=url, headers=headers)
    total = response.json()["count"]
    if page_number > total:
        break
    page_number += 100
    page_generator = my_generator(response.json()["data"])

    for fund in page_generator:
        fund_info = {
            "id": fund["id"],
            "name": fund["name"],
            "location": fund["location"],
            "type": fund["type"],
            "tier": fund["tier"],
            }
        funds_info.append(fund_info)
    print(f"Funds from page {int(page_number/100)} added.")

In [47]:
len(funds_info)

8934

Super. Now we have the data for all the funds. Let's move on. For the next step, we need to get the fund id to get the portfolio.
Now the goal is to collect information about all unique coins in the portfolios.
In the coin data I found a list called "fundIds". This will make the work easier.
There may also be funds or coins with no data =/

There is also an option to separate coins by life cycle: crowdsale,inactive,funding,scheduled,traded. I'll play with this later but for now traded coins are more interesting.

In [62]:
coins = list()
names = set()
funds_generator = my_generator(funds_info)
for index, fund in enumerate(funds_generator):
    start = time()
    print(f"#{index+1} Fund '{fund['name']}' id:{fund['id']} in work!")
    id = fund["id"]
    url = f"https://api.cryptorank.io/v0/coins?fundIds={id}&withFundingRoundsData=true&lifeCycle=traded&locale=en"
    response = requests.get(url=url, headers=headers)
    # TODO Try to change it with try: except:
    if len(response.json()["data"]) != 0:
        portfolio = my_generator(response.json()["data"])
        count = 0
        for coin in portfolio:
            if coin["name"] not in names:
                names.add(coin["name"])
                coin = {
                    "name": coin["name"],
                    "symbol": coin["symbol"],
                    "type": coin["type"],
                    "total_funds": len(coin["fundIds"]),
                }
                coins.append(coin)
                count += 1
        end = time()
        print(f"{count} coins added. Total: {len(coins)}.")
        sleep(1)
    execution_time = end - start
    if (index+1) % 50 == 0:
        sleep(7)
    print(f"Collect time: {execution_time}. ")
    

#1 Fund 'Coinbase Ventures' id:5 in work!
110 coins added. Total: 110.
Collect time: 1.0997488498687744. 
#2 Fund 'Andreessen Horowitz (a16z)' id:57 in work!
29 coins added. Total: 139.
Collect time: 1.2154717445373535. 
#3 Fund 'Pantera Capital' id:7 in work!
50 coins added. Total: 189.
Collect time: 0.898066520690918. 
#4 Fund 'Polychain Capital' id:43 in work!
23 coins added. Total: 212.
Collect time: 2.817453384399414. 
#5 Fund 'HashKey Capital' id:63 in work!
54 coins added. Total: 266.
Collect time: 0.8700911998748779. 
#6 Fund 'DragonFly Capital' id:40 in work!
15 coins added. Total: 281.
Collect time: 1.3152220249176025. 
#7 Fund 'Binance Labs' id:38 in work!
84 coins added. Total: 365.
Collect time: 2.1606814861297607. 
#8 Fund 'Multicoin Capital' id:42 in work!
22 coins added. Total: 387.
Collect time: 3.914524793624878. 
#9 Fund 'Balaji Srinivasan' id:372 in work!
3 coins added. Total: 390.
Collect time: 1.2237110137939453. 
#10 Fund 'Galaxy' id:62 in work!
8 coins added. To

In [59]:
coins_1 = coins.copy()
coins_1

[{'name': 'Polygon', 'symbol': 'MATIC', 'type': 'coin', 'total_funds': 42},
 {'name': 'Uniswap', 'symbol': 'UNI', 'type': 'token', 'total_funds': 11},
 {'name': 'Near Protocol',
  'symbol': 'NEAR',
  'type': 'coin',
  'total_funds': 36},
 {'name': 'Lido DAO Token',
  'symbol': 'LDO',
  'type': 'token',
  'total_funds': 26},
 {'name': 'Aptos', 'symbol': 'APT', 'type': 'coin', 'total_funds': 19},
 {'name': 'Immutable', 'symbol': 'IMX', 'type': 'token', 'total_funds': 23},
 {'name': 'Optimism', 'symbol': 'OP', 'type': 'token', 'total_funds': 4},
 {'name': 'The Graph', 'symbol': 'GRT', 'type': 'token', 'total_funds': 11},
 {'name': 'Celestia', 'symbol': 'TIA', 'type': 'token', 'total_funds': 31},
 {'name': 'Arbitrum', 'symbol': 'ARB', 'type': 'token', 'total_funds': 8},
 {'name': 'Terra Classic',
  'symbol': 'LUNC',
  'type': 'coin',
  'total_funds': 18},
 {'name': 'Flow', 'symbol': 'FLOW', 'type': 'coin', 'total_funds': 12},
 {'name': 'Synthetix Network',
  'symbol': 'SNX',
  'type': 'tok